In [1]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 02-27 11:05:43 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.999 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-unsloth-bnb-4bit with actual GPU utilization = 73.23%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 12.0 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.45 GB. Also swap space = 4 GB.
INFO 02-27 11:06:45 config.py:549] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaul

[W227 11:06:51.341021081 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-27 11:06:52 loader.py:1089] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 02-27 11:06:52 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-27 11:08:08 model_runner.py:1115] Loading model weights took 1.3190 GB
INFO 02-27 11:08:08 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-27 11:08:13 worker.py:267] Memory profiling takes 4.62 seconds
INFO 02-27 11:08:13 worker.py:267] the current vLLM instance can use total_gpu_memory (12.00GiB) x gpu_memory_utilization (0.73) = 8.79GiB
INFO 02-27 11:08:13 worker.py:267] model weights take 1.32GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 6.39GiB.
INFO 02-27 11:08:13 executor_base.py:111] # cuda blocks: 14957, # CPU blocks: 9362
INFO 02-27 11:08:13 executor_base.py:116] Maximum concurrency for 1024 tokens per request: 233.70x
INFO 02-27 11:08:14 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:19<00:00,  1.37it/s]

INFO 02-27 11:08:34 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.84 GiB
INFO 02-27 11:08:34 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 25.42 seconds



Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
tokenizer.chat_template = """{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>\n' }}{% elif message['role'] == 'assistant' %}{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>\n' }}{% elif message['role'] == 'system' %}{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>\n' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"""

In [4]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [5]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 20,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = 512,
    max_completion_length = 256,
    num_train_epochs = 1, # Set to 1 for a full training run
    # max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [6]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train(resume_from_checkpoint = True)

/mnt/d/WSL2/Ubuntu/home/yuiseki/miniconda_envs/unsloth/lib/python3.11/site-packages/transformers/trainer.py:3423: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func


TrainOutput(global_step=7473, training_loss=0.0, metrics={'train_runtime': 0.0068, 'train_samples_per_second': 1102304.688, 'train_steps_per_second': 1102304.688, 'total_flos': 0.0, 'train_loss': 0.0})

In [7]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:10<00:00, 10.00s/it, est. speed input: 1.60 toks/s, output: 100.86 toks/s]


"How many r's are in strawberry?赸\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assistant\n赸assi

In [8]:
model.save_lora("grpo_saved_lora")

In [9]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.30s/it, est. speed input: 4.62 toks/s, output: 105.59 toks/s]


'猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞'

In [10]:
checkpoint_path = "outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K"
model.save_pretrained(checkpoint_path)
tokenizer.save_pretrained(checkpoint_path)

('outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K/tokenizer_config.json',
 'outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K/special_tokens_map.json',
 'outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K/vocab.json',
 'outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K/merges.txt',
 'outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K/added_tokens.json',
 'outputs/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K/tokenizer.json')

In [ ]:
model.push_to_hub_gguf(
        "yuiseki/YuisekinAI-Qwen2.5-1.5B-GRPO-GSM8K-GGUF",
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )